In [281]:
from sympy import symbols, pprint
from sympy import diff
from sympy.solvers import solve
import numpy as np
from scipy import optimize
import string
import random
from autodp.transformer_zoo import Composition
from functools import lru_cache

# data subject
class Entity():
    def __init__(self, name="", id=None):
        self.name = name
        self.id = id

scalar_name2obj = {}

In [282]:
def individual_RDP_gaussian(params, alpha):
    """
    :param params:
        'sigma' --- is the normalized noise level: std divided by global L2 sensitivity
    :param alpha: The order of the Renyi Divergence
    :return: Evaluation of the RDP's epsilon
    """
    sigma = params['sigma']
    value = params['value']
    L = params['L']
    assert(sigma > 0)
    assert(alpha >= 0)
    
    return (alpha * (L ** 2) * (value**2)) / (2 * (sigma**2))

import math

from autodp.autodp_core import Mechanism
from autodp import rdp_bank, dp_bank, fdp_bank, utils
from autodp import transformer_zoo

from scipy.optimize import minimize_scalar


# Example of a specific mechanism that inherits the Mechanism class
class iDPGaussianMechanism(Mechanism):
    def __init__(self, sigma, value, L, entity, name='Gaussian',
                 RDP_off=False, approxDP_off=False, fdp_off=True,
                 use_basic_RDP_to_approxDP_conversion=False,
                 use_fDP_based_RDP_to_approxDP_conversion=False):
        # the sigma parameter is the std of the noise divide by the l2 sensitivity
        Mechanism.__init__(self)

        self.name = name # When composing
        self.params = {'sigma': sigma, 'value':value, 'L':L} # This will be useful for the Calibrator
        self.entity = entity
        # TODO: should a generic unspecified mechanism have a name and a param dictionary?

        self.delta0 = 0
        if not RDP_off:
            new_rdp = lambda x: individual_RDP_gaussian({'sigma': sigma,
                                                        'value': value,
                                                        'L':L}, x)
            if use_fDP_based_RDP_to_approxDP_conversion:
                # This setting is slightly more complex, which involves converting RDP to fDP,
                # then to eps-delta-DP via the duality
                self.propagate_updates(new_rdp, 'RDP', fDP_based_conversion=True)
            elif use_basic_RDP_to_approxDP_conversion:
                self.propagate_updates(new_rdp, 'RDP', BBGHS_conversion=False)
            else:
                # This is the default setting with fast computation of RDP to approx-DP
                self.propagate_updates(new_rdp, 'RDP')

        if not approxDP_off: # Direct implementation of approxDP
            new_approxdp = lambda x: dp_bank.get_eps_ana_gaussian(sigma, x)
            self.propagate_updates(new_approxdp,'approxDP_func')

        if not fdp_off: # Direct implementation of fDP
            fun1 = lambda x: fdp_bank.log_one_minus_fdp_gaussian({'sigma': sigma}, x)
            fun2 = lambda x: fdp_bank.log_neg_fdp_grad_gaussian({'sigma': sigma}, x)
            self.propagate_updates([fun1,fun2],'fDP_and_grad_log')
            # overwrite the fdp computation with the direct computation
            self.fdp = lambda x: fdp_bank.fDP_gaussian({'sigma': sigma}, x)

        # the fDP of gaussian mechanism is equivalent to analytical calibration of approxdp,
        # so it should have been automatically handled numerically above


        # Discussion:  Sometimes delta as a function of eps has a closed-form solution
        # while eps as a function of delta does not
        # Shall we represent delta as a function of eps instead?

In [283]:
class AdversarialAccountant():

    def __init__(self, max_budget=10, delta=1e-6):
        self.entity2ledger = {}
        self.max_budget = max_budget
        self.delta = delta

    def append(self, entity2mechanisms):
        for key, ms in entity2mechanisms.items():
            if key not in self.entity2ledger.keys():
                self.entity2ledger[key] = list()
            for m in ms:
                self.entity2ledger[key].append(m)

    def get_eps_for_entity(self, entity_name):
        # compose them with the transformation: compose.
        compose = Composition()
        mechanisms = self.entity2ledger[entity_name]
        composed_mech = compose(mechanisms, [1]*len(mechanisms))

        # Query for eps given delta
        return Scalar(value=composed_mech.get_approxDP(self.delta), 
                      min_val=0, 
                      max_val=self.max_budget,
                      ent=Entity(name=entity_name))

    def has_budget(self, entity_name):
        return self.get_eps_for_entity(entity_name)._value < self.max_budget
    
    @property
    def entities(self):
        return self.entity2ledger.keys()
    
    @property
    def overbudgeted_entities(self):
        entities = set()
        
        for ent in self.entities:
            if not self.has_budget(ent):
                entities.add(ent)
                
        return entities
    
    def print_ledger(self, delta=1e-6):
        for entity, mechanisms in self.entity2ledger.items():
            print(entity + "\t" + str(self.get_eps_for_entity(entity)._value))

In [450]:
def run(func, **kwargs):
    return func.subs(kwargs)

def run_specific(f, **kwargs):
    """pass in kwargs to run in fixed polynomial because this is what
    optimize.brute expects"""
    return run(f, **kwargs)


@lru_cache(maxsize=None)
def search(run_specific_args, rranges, full_output, finish, disp):
    return optimize.shgo(run_specific_args, rranges)

class Scalar():
    
    def __init__(self, value, min_val=None, max_val=None, poly=None, ent=None, name=None):
        
        if name is None:
            lower_upper_alphabet = string.ascii_letters
            name = ''.join([random.choice(lower_upper_alphabet) for i in range(5)])
        
        self.name = name
        self._value = value
        self._min_val = min_val
        self._max_val = max_val
        self.enabled = True
        
        if poly is not None:
            # if this Scalar is being formed as a function of other Scalar objects
            self._poly = poly
            
        elif ent is not None:
            # if you're creating a Scalar for the first time (no parents)
            self.scalar_name = self.name + "_" + ent.name
            self._poly = symbols(self.scalar_name)
            scalar_name2obj[self.scalar_name] = self            
        else:
            raise Exception("Poly or ent must be not None")

    @property
    def poly(self):
        return self._poly
 
    @property
    def value(self):
        
        if(self._value is not None):
            return self._value
        
        sy_names = self.poly.free_symbols
        sym = list()
        for sy_name in sy_names:
            sym.append(scalar_name2obj[str(sy_name)])

        run_specific_args, index2symbol, symbol2index = self.create_run_specific_args(f=self.poly)

        inputs = list()

        for sym in index2symbol:
            inputs.append(scalar_name2obj[sym]._value)
        return run_specific_args(inputs)
            
    @property
    def min_val(self):
        return self._min_val
    
    @property
    def max_val(self):    
        return self._max_val
    
    def __mul__(self, other):
        
        result_poly = self.poly * other.poly
        
        result = Scalar(value=None, poly=result_poly)
        
        return result
    
    def __add__(self, other):
        result_poly = self.poly + other.poly
        
        result = Scalar(value=None, poly=result_poly)
        
        return result
    
    def __sub__(self, other):
        result_poly = self.poly - other.poly
        
        result = Scalar(value=None, poly=result_poly)
        
        return result
    
    def __str__(self):
        return str(self.poly) + "=" + str(self.value)
    
    def __repr__(self):
        return str(self)

    @property
    def sens(self):
        if self.min_val is not None and self.max_val is not None:
            return self.max_val - self.min_val

    def neg_deriv(self, name):
        obj = scalar_name2obj[name]
        return -diff(self.poly, obj.poly)
        

    def create_run_specific_args(self, f):

        free_symbols_list = list(self.poly.free_symbols)
        index2symbol = list(map(lambda x:str(x), free_symbols_list))

        symbol2index = {}
        for i,sym in enumerate(index2symbol):
            symbol2index[sym] = i

        def _run_specific_args(tuple_of_args, *params):
            kwargs = {}
            for sym,i in symbol2index.items():
                kwargs[sym] = tuple_of_args[i]

            return run_specific(f=f, **kwargs)

        return _run_specific_args, index2symbol, symbol2index
    
    def get_mechanism(self,
            symbol_name = 'b',
            sigma = 0.1):
        
        # Step 1: get derivative we want to maximize
        z = self.neg_deriv(symbol_name)

        # Step 2: prepare metadata for optimize.brute() function
        sy_names = z.free_symbols
        sym = list()
        for sy_name in sy_names:
            sym.append(scalar_name2obj[str(sy_name)])

        run_specific_args, index2symbol, symbol2index = self.create_run_specific_args(f=z)

        rranges = list()
        for i,sym in enumerate(index2symbol):
            obj = scalar_name2obj[sym]
            rranges.append(tuple([obj.min_val, obj.max_val]))

        # Step 3: maximize the derivative over a bounded range of <entity_name>
        resbrute = search(run_specific_args, tuple(rranges), full_output=False, finish=None, disp=True)
        resbrute = resbrute.x
        
        if isinstance(resbrute, np.float64):
            L = resbrute
        else:
            L = resbrute[symbol2index[symbol_name]]
        input_obj = scalar_name2obj[symbol_name]

        # Step 4: create the gaussian mechanism object
        gm1 = iDPGaussianMechanism(sigma=sigma, value=input_obj._value, L=L, entity=symbol_name.split("_")[1], name='gm_'+symbol_name)

        return gm1
    
    def get_all_entity_mechanisms(self,sigma=0.1):
        sy_names = self.poly.free_symbols
        entity2mechanisms = {}
        for sy_name in sy_names:
            mechanism = self.get_mechanism(symbol_name=str(sy_name), sigma=sigma)
            
            split_name = str(sy_name).split("_")
            entity_name = split_name[1]
            var_name = split_name[0]
            
            if entity_name not in entity2mechanisms.keys():
                entity2mechanisms[entity_name] = list()
            
            entity2mechanisms[entity_name].append(mechanism)
            
            
        return entity2mechanisms
    
    @property
    def entities(self):
        entities = set()
        sy_names = self.poly.free_symbols 
        for sy_name in sy_names:
            entities.add(str(sy_name).split("_")[1])
        return entities
    
    def publish(self, acc, sigma=1.5):
        acc_original = acc
        
        assert sigma > 0
        
        acc_temp = deepcopy(acc_original)
        
        # get mechanisms for new publish event
        ms = self.get_all_entity_mechanisms(sigma=sigma)
        acc_temp.append(ms)
        
        overbudgeted_entities = acc_temp.overbudgeted_entities
        
        sample = random.gauss(0,sigma)                
        
        while len(overbudgeted_entities) > 0:

            for sy_name in self.poly.free_symbols:
                entity_name = str(sy_name).split("_")[1]
                if(entity_name in overbudgeted_entities):
                    sym = scalar_name2obj[str(sy_name)]
                    self._poly = self.poly.subs(sym.poly, 0)
                    
            
            acc_temp = deepcopy(acc_original)
            
            # get mechanisms for new publish event
            ms = self.get_all_entity_mechanisms(sigma=sigma)
            acc_temp.append(ms)
            
            overbudgeted_entities = acc_temp.overbudgeted_entities
        

        output = self.value + sample
        
        acc_original.entity2ledger = deepcopy(acc_temp.entity2ledger)
        
        return output
   

In [451]:
from copy import deepcopy

In [462]:
bob = Scalar(value=1, min_val=-2, max_val=2, ent=Entity(name="Bob"))
bobby = Scalar(value=1, min_val=-2, max_val=2, ent=Entity(name="Bob"))
alice = Scalar(value=1, min_val=-1, max_val=1, ent=Entity(name="Alice"))
charlie = Scalar(value=2, min_val=-2, max_val=2, ent=Entity(name="Charlie"))
david = Scalar(value=2, min_val=-2, max_val=2, ent=Entity(name="David"))

acc = AdversarialAccountant(max_budget=70)

# PhiScalar()
bob2 = bob + bob
bobby2 = bobby + bobby

alice2 = alice + alice
charlie2 = charlie + charlie

(alice2 * bobby2) + (alice2 * bob2)

def __add__(self, other):
    if self.gama==False and other.gamma== False and self.entity == other.entity
        return Scalar(entity=self.entity,
                      value=self.value + other.value, 
                      min_val=self.min_val + other.min_value,
                      max_val=self.max_val + other.max_val,
                      symbol_name=entity_name + "_" + random_hash)
    else:
        
        self.gama = True
        self.poly = symbol(self.symbol_name)
        
        other.gama = True
        other.poly = symbol(other.symbol_name)
        
        


# GammaScalar()
out = bob2**2 + alice2*0.5 + bobby2**3

# 

In [463]:
%%timeit -n1 -r1

public_out = out.publish(acc=acc, sigma=0.5)
acc.print_ledger()

Bob	27.811968501910776
Alice	11.688596249354894
45.5 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [403]:
result = optimize.shgo(eggholder, bounds)

In [404]:
result.x, result.fun

(array([439.48099854, 453.9774378 ]), -935.3379515605761)